## Imports

In [1]:
CONST_TEST_MODE = 0

In [2]:
# imports
import numpy as np
import pandas as pd
import calendar

import matplotlib.pyplot as plt
import seaborn

import platform
import sys
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from pandas.tools.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from scipy.stats.stats import pearsonr

import numbers

from pandas import Series
import datetime as dt

# Multiple dataframes in single cell
%matplotlib inline

import seaborn as sns

# Auto size the plots to be able to see better
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

CONST_STATIONARY = 'STATIONARY'
CONST_NON_STATIONARY = 'NON_STATIONARY'

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Data Loading, Merging and Massaging

In [3]:
# Function to return the load of all data - used across mulitiple notebooks as I try different scripts.
# This will load, format the data, and then add any additional columns or features
# This was originally done in another notebook, and also loaded into SQL for data merge checking
def fn_load_all_data(loadDummyVariables=0):

    # Working Dir / Load Data - in case we need to distribute later on
    if (platform.system() == 'Windows'):
        currentOS = 'Windows'
        workDir = 'D:\\project\\data\\kg_jpn_rest\\'
    # MAC
    elif (platform.system() == 'Darwin'):
        currentOS = 'Mac'
        workDir = '//Project/data/kg_jpn_rest/'
    # AWS
    elif (platform.system() == 'Linux'):
        currentOS = 'Linux'
        workDir = '//data/'

    print('Data Load - Loading data on: ' + currentOS + ' Directory: ' + workDir)

    # reservations in the air system
    pd_air_reserve = pd.read_csv(workDir + 'air_reserve.csv', parse_dates=(['visit_datetime','reserve_datetime']), infer_datetime_format=True)
    pd_air_reserve.name = 'pd_air_reserve'
    # reservation in the hpg system
    pd_hpg_reserve = pd.read_csv(workDir + 'hpg_reserve.csv', parse_dates=(['visit_datetime','reserve_datetime']), infer_datetime_format=True)
    pd_hpg_reserve.name = 'pd_hpg_reserve'
    # contains info about the store info.  lat and long is the area which the store belongs
    pd_air_store_info = pd.read_csv(workDir + 'air_store_info.csv', infer_datetime_format=True)
    pd_air_store_info.name = 'pd_air_store_info'
    # contains info about select air restaraunts.  Lat and long is the area where store belongs
    pd_hpg_store_info = pd.read_csv(workDir + 'hpg_store_info.csv', infer_datetime_format=True)
    pd_hpg_store_info.name = 'pd_hpg_store_info'
    # file contains HISTORICAL visit data for the air restaraunts
    pd_air_visit_data = pd.read_csv(workDir + 'air_visit_data.csv', parse_dates=(['visit_date']), infer_datetime_format=True)
    pd_air_visit_data.name = 'pd_air_visit_data'
    # give basic info about the calendar dates in the dataset
    pd_date_info = pd.read_csv(workDir + 'date_info.csv', parse_dates=(['calendar_date']), infer_datetime_format=True)
    pd_date_info.name = 'pd_date_info'
    # allows you to join select restaraunts that have both air and hpg systems
    pd_store_id_relation = pd.read_csv(workDir + 'store_id_relation.csv', infer_datetime_format=True)
    pd_store_id_relation.name = 'pd_store_id_relation'

    # Flatten the hpg reserve and air reserve
    # add a date
    pd_hpg_reserve.sort_values(['hpg_store_id','visit_datetime'], ascending=[True,True], inplace=True)
    pd_hpg_reserve['visit_date'] = pd_hpg_reserve['visit_datetime'].dt.date
    hpgPivot = pd.pivot_table(pd_hpg_reserve, 
                              values='reserve_visitors', 
                              index=['hpg_store_id','visit_date'],
                              aggfunc=np.sum)
    hpgPivot.reset_index(inplace=True)
    hpgPivot['visit_date'] = hpgPivot['visit_date'].astype('datetime64[ns]')
    hpgPivot.head()


    pd_air_reserve.sort_values(['air_store_id','visit_datetime'], ascending=[True,True], inplace=True)
    pd_air_reserve['visit_date'] = pd_air_reserve['visit_datetime'].dt.date
    airPivot = pd.pivot_table(pd_air_reserve, 
                              values='reserve_visitors', 
                              index=['air_store_id','visit_date'],
                              aggfunc=np.sum)
    airPivot.reset_index(inplace=True)
    airPivot['visit_date'] = airPivot['visit_date'].astype('datetime64[ns]')
    airPivot.head()

    # First set the superjoin to the air_visit_data
    dfSuper = pd_air_visit_data

    
    print('Data Load - Merging Data')
    # join visit + store xmap
    dfSuper = pd.merge(dfSuper, pd_store_id_relation, how='left', on ='air_store_id')
    # join hpg store info
    dfSuper = pd.merge(dfSuper, pd_hpg_store_info, how='left', on='hpg_store_id')
    # join air store info
    dfSuper = pd.merge(dfSuper, pd_air_store_info, how='left', on='air_store_id')
    # now the reservations
    dfSuper = pd.merge(dfSuper, hpgPivot, how='left', on=['hpg_store_id','visit_date'])
    dfSuper = pd.merge(dfSuper, airPivot, how='left', on=['air_store_id','visit_date'])
    # now the holiday
    dfSuper = pd.merge(dfSuper, pd_date_info, how='left', left_on='visit_date', right_on='calendar_date')

    # rename the columns
    dfSuper.rename(columns={'latitude_x':'hpg_latitude', 'longitude_x':'hpg_longitude', 
                           'latitude_y':'air_latitude', 'longitude_y':'air_longitude',
                           'reserve_visitors_x':'hpg_reserve_visitors', 'reserve_visitors_y':'air_reserve_visitors' }, inplace=True)

    # Set any numbers that are na
    dfSuper['visitors'].replace({np.nan: 0}, inplace=True)
    dfSuper['air_latitude'].replace({np.nan: 0}, inplace=True)
    dfSuper['air_longitude'].replace({np.nan: 0}, inplace=True)
    dfSuper['hpg_latitude'].replace({np.nan: 0}, inplace=True)
    dfSuper['hpg_longitude'].replace({np.nan: 0}, inplace=True)
    dfSuper['air_reserve_visitors'].replace({np.nan: 0}, inplace=True)
    dfSuper['hpg_reserve_visitors'].replace({np.nan: 0}, inplace=True)

    # blank strings
    dfSuper['air_genre_name'].replace({np.nan: ''}, inplace=True)
    dfSuper['air_area_name'].replace({np.nan: ''}, inplace=True)
    dfSuper['hpg_genre_name'].replace({np.nan: ''}, inplace=True)
    dfSuper['hpg_area_name'].replace({np.nan: ''}, inplace=True)

    

    # now if we have restaranut info data between the two reservation (AIR + HPG) systems for the same restaraunt, 
    #   take the AIR as the tie-breaker
    # will take in two columns.  If 1 has data present, use that, else if col 2 has data, return that
    def fn_assign_air_hpg_value(colAir, colHpg):

        # if a number
        if isinstance(colAir, numbers.Number) and isinstance(colHpg, numbers.Number):
            if colAir > 0:
                return colAir
            elif colHpg > 0:
                return colHpg
            else:
                return 0;

        # if it's a string
        else:
            if len(colAir) > 0:
                return colAir
            elif len(colHpg) > 0:
                return colHpg
            else:
                return ''

    # Define if it is a weekend (Friday, Sat, Sun)
    def fn_is_weekend(col):
        if ((col==4) or (col==5) or (col==6)):
            return 1
        else:
            return 0


    # Start to combine
    dfSuper['genre_name	'] = dfSuper.apply(lambda row: fn_assign_air_hpg_value(row['air_genre_name'], row['hpg_genre_name']), axis=1)
    dfSuper['area_name'] = dfSuper.apply(lambda row: fn_assign_air_hpg_value(row['air_area_name'], row['hpg_area_name']), axis=1)
    dfSuper['latitude'] = dfSuper.apply(lambda row: fn_assign_air_hpg_value(row['air_latitude'], row['hpg_latitude']), axis=1)
    dfSuper['longitude'] = dfSuper.apply(lambda row: fn_assign_air_hpg_value(row['air_longitude'], row['hpg_longitude']), axis=1)

    # Add the reservations between the two systems
    dfSuper['reserve_visitors'] = dfSuper['hpg_reserve_visitors'] + dfSuper['air_reserve_visitors']

    print('Data Load - Adding Features')
    # Features
    # Add day and month
    dfSuper['month_num'] = dfSuper['visit_date'].dt.month
    dfSuper['month_name'] = dfSuper['month_num'].apply(lambda x: calendar.month_abbr[x])
    
    dfSuper['dayofmonth_num'] = dfSuper['visit_date'].dt.day
    dfSuper['dayofweek_num'] = dfSuper['visit_date'].dt.dayofweek  # Monday is 0, Sunday is 6

    # Weekend
    dfSuper['weekend'] = dfSuper.apply(lambda row: fn_is_weekend(row['dayofweek_num']), axis=1)

    # Correlations
    # reservations to visitors correlation
    dfCorr = dfSuper.copy()
    # shorten the list
    dfCorr = dfCorr[['air_store_id','visitors','reserve_visitors']]
    # re-create dataframe from correlation grouped by the air_store_id
    dfCorr = dfCorr.groupby('air_store_id')[['visitors','reserve_visitors']].corr().iloc[0::2]
    # rename the colum
    dfCorr = dfCorr.rename(columns={'reserve_visitors':'corr_vis_resv'})
    # drop the "visitors" columns that are left over from the matrix
    dfCorr.reset_index(inplace=True)
    dfCorr.drop(['visitors','level_1'], axis=1, inplace=True)
    # set any nan to 0
    dfCorr['corr_vis_resv'].replace({np.nan: 0}, inplace=True)
    # join back
    dfSuper= pd.merge(dfSuper, dfCorr, how='left', on ='air_store_id', )
    dfSuper.rename(columns={'corr_vis_resv_x':'corr_vis_resv'}, inplace=True)

    
    # Drop the duplicated columns
    dfSuper.drop(['hpg_store_id','hpg_genre_name','hpg_area_name','hpg_latitude','hpg_longitude','air_genre_name',
                   'air_area_name','air_latitude','air_longitude', 'hpg_reserve_visitors','air_reserve_visitors'], axis=1, inplace=True)

    # Set the indexes for the data
    # Set Indexes
    dfSuper.set_index('visit_date', inplace=True)

    # Set any floats for the data (to work with time-series)
    # Set data types
    dfSuper['visitors'] = dfSuper['visitors'].astype('float32')

    
    # do each one in a consolidated
    def convert_dummy_vars(dataFrame, columnName, dropFirst=False, colRename=''):

        temp = pd.get_dummies(dataFrame[columnName], drop_first=dropFirst)

        if colRename != '':
            temp.columns = [(colRename)]

        #dataFrame.drop([columnName], axis=1, inplace=True)
        dataFrame = pd.concat([dataFrame, temp], axis=1)    
        return dataFrame
    
    # Hit the dummy variables
    if loadDummyVariables==1:
        dfSuper = convert_dummy_vars(dataFrame=dfSuper, columnName='day_of_week')
        dfSuper = convert_dummy_vars(dataFrame=dfSuper, columnName='month_name')
        
    
    # Create columns and default (initialize)
    dfSuper['forecast'] = 0
    dfSuper['forecast'] = dfSuper['forecast'].astype('float32')    
        
    print('Data Load - Finished')
    
    # Final Re-Organization of columns
    
    return dfSuper

In [4]:
# Test the function above
if CONST_TEST_MODE==1:
    df = fn_load_all_data(1)
# df.head()

In [5]:
# df.head()

In [6]:
# Reorgder the columns
#df = df[['air_store_id','visitors','reserve_visitors','corr_vis_resv','genre_name','area_name','latitude','longitude',\
#    'calendar_date','holiday_flg','month_num','month_name','dayofmonth_num','day_of_week','dayofweek_num','weekend',\
#    'forecast']]

In [7]:
# do each one in a consolidated
def convert_dummy_vars(dataFrame, columnName, dropFirst=False, colRename=''):
    
    temp = pd.get_dummies(dataFrame[columnName], drop_first=dropFirst)
    
    if colRename != '':
        temp.columns = [(colRename)]
    
    #dataFrame.drop([columnName], axis=1, inplace=True)
    dataFrame = pd.concat([dataFrame, temp], axis=1)    
    return dataFrame

In [8]:
'''
ben = df.copy()
ben.head()

# CHOOSE WHICH FEATURES TO INCLUDE
# 'amount_tsh', - leave out - add after binning and plotting

shortList_v1 = ['day_of_week','month_name']
ben = convert_dummy_vars(dataFrame=ben, columnName='day_of_week')
ben = convert_dummy_vars(dataFrame=ben, columnName='month_name')
'''

"\nben = df.copy()\nben.head()\n\n# CHOOSE WHICH FEATURES TO INCLUDE\n# 'amount_tsh', - leave out - add after binning and plotting\n\nshortList_v1 = ['day_of_week','month_name']\nben = convert_dummy_vars(dataFrame=ben, columnName='day_of_week')\nben = convert_dummy_vars(dataFrame=ben, columnName='month_name')\n"

## EDA / Plot Functions

In [9]:
# FUNCTION
def fn_ts_add_cycletrend_analysis(df, colName):
        
    # cycle and trend
    cycle, trend = sm.tsa.filters.hpfilter(df[colName])
    df['cycle'] = cycle
    df['trend'] = trend
    
    # Simple moving average
    df['3-day-SMA'] = df[colName].rolling(window=3).mean()
    df['7-day-SMA'] = df[colName].rolling(window=7).mean()
    df['14-day-SMA'] = df[colName].rolling(window=14).mean()
    df['31-day-SMA'] = df[colName].rolling(window=31).mean()
    
    # EWMA
    df['EWMA_7_days'] = df[colName].ewm(span=7).mean()
    df['EWMA_14_days'] = df[colName].ewm(span=14).mean()
    df['EWMA_31_days'] = df[colName].ewm(span=31).mean()
    
    newColList = ['cycle','trend','3-day-SMA','7-day-SMA','14-day-SMA','31-day-SMA',
                 'EWMA_7_days','EWMA_14_days','EWMA_31_days']
    
    return df, newColList

In [10]:
# Testing
#df = fn_ts_add_cycletrend_analysis(df, 'visitors')
#df.plot()

In [11]:
# Ad-fuller Check
def fn_adf_check(time_series):
    """
    Pass in a time series, returns ADF report
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
        return CONST_STATIONARY
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
        return CONST_NON_STATIONARY

In [12]:
def fn_plot_auto_corr(df):
    autocorrelation_plot(df.dropna())

In [13]:
def fn_plot_acf(df):
    plot = plot_acf(df.dropna())

In [14]:
def fn_plot_pacf(df):
    plot = plot_pacf(df.dropna())

In [15]:
def fn_plot_decomposition(series, frequency):    
    decomposition1 = seasonal_decompose(series, freq=frequency)
    fig = plt.figure()
    fig = decomposition1.plot()
    fig.set_size_inches(20,8)

In [16]:
def fn_plot_result_diagnostics(model_results):
    fPlot = model_results.plot_diagnostics(figsize=(10,8))

In [17]:
if CONST_TEST_MODE==1:
    subDF = df[df['air_store_id']=='air_1c0b150f9e696a5f']
    
    fn_plot_auto_corr(subDF['visitors'])
    fn_plot_acf(subDF['visitors'])
    fn_plot_pacf(subDF['visitors'])
    fn_plot_decomposition(subDF['visitors'])
    

# Models to Run

In [ ]:
# Not yet tested / implemented
def fn_run_arima_timeseries(visitMergeDF, storeToEvaluate, orderList, logger):
    
    errorOccured = False
    
    if GLOBAL_CONST_DEBUG ==1:
        print('Start Processing Restaraunt: ' + str(storeToEvaluate))
    
    # create ts sub-set
    ts = visitMergeDF[visitMergeDF['air_store_id']==storeToEvaluate].copy()
    ts.asfreq('D')
    
    minDate = ts.index.min()
    
    # add dates to predict
    idx = pd.DataFrame(pd.date_range('2017-04-23','2017-05-31'), columns={'dateRange'})
    idx.set_index('dateRange',inplace=True)
    ts = pd.concat([ts,idx], axis=1)

    # Impute
    ts['air_store_id'].replace({np.nan: storeToEvaluate}, inplace=True)
    ts['visitors'].replace({np.nan: 0}, inplace=True)
    ts['visitors_log'].replace({np.nan: 0}, inplace=True)
    
    # Try Catch Here
    try:
    
        # standard ARIMA model with order list passed in
        model = ARIMA(ts['visitors'][minDate:"2017-04-23"], order=orderList)
        modelFit = model.fit(disp=-1)
        # 39 days is how far out we are predicting
        results = modelFit.forecast(39)  
        # set forecast
        ts['forecast']["2017-04-23":"2017-05-31"] = results[0][:]

        if GLOBAL_CONST_DEBUG ==1:
            print('Start Processing Restaraunt - LOG: ' + str(storeToEvaluate))

        # Do the log while we are in here
        modelLog = ARIMA(ts['visitors_log'][minDate:"2017-04-23"], order=orderList)
        modelLog_fit = modelLog.fit(disp=-1)
        resultsLog = modelLog_fit.forecast(39)
        ts['forecast_log']["2017-04-23":"2017-05-31"] = resultsLog[0][:]
        ts['forecast_logExp'] = np.exp(ts['forecast_log'])  # Revert log back to standard
    
    
    except: 
        #(RuntimeError, TypeError, NameError):
        #print('Error')
        #print(RuntimeError)
        #print(TypeError)
        #print(NameError)
        #print('\n')
        logger.info('Error processing store: ' + storeToEvaluate)
        errorOccured = True
           
    pass


    if GLOBAL_CONST_DEBUG ==1:
        print('Finished Processing Restaraunt: ' + str(storeToEvaluate))


    # return back only what we predicted
    dfTSReturn = ts[:]["2017-04-23":]
    return dfTSReturn, errorOccured

## Finalization of Submission Files

In [18]:
# This function will finalize the submission file
def fn_finalize_submission_file(exportDir, colNameToSubmit):
    
    arimaExportDir = 'D:\\project\\data\\kg_jpn_rest\\export\\'
    arimaExportFile = 'export_results.csv'

    armRsltDF = pd.read_csv(arimaExportDir + arimaExportFile)
    pd_sample_submission = pd.read_csv('D:\\project\\data\\kg_jpn_rest\\' + 'sample_submission.csv')
    
    # Concatenate the fields to create the ID
    armRsltDF['id'] = ''
    armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['air_store_id'])
    armRsltDF['id'] = armRsltDF['id'] + '_'
    armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['visit_date'].astype(str))

    # Drop the visitors (will rename later)
    armRsltDF.drop('visitors', axis=1, inplace=True)

    # Must turn any negatives into a zero
    # Not sure why this needs to come after, perhaps with the merge it throws some things between float/int offf
    def fn_set_negative_to_zero(col):
        if col < 0:
            return 0.0
        else:
            return col

    # Took out the log items for now    
    armRsltDF['forecast'] = armRsltDF.apply(lambda row: fn_set_negative_to_zero(row['forecast']), axis=1)
    armRsltDF[armRsltDF['forecast']<0]['forecast'] = 0.0

    # Join to sample submission
    armRsltDF = pd.merge(pd_sample_submission, armRsltDF, how='inner', on=('id'))

    # Drop the visitors (will rename later)
    armRsltDF.drop('visitors', axis=1, inplace=True)

    # 32019 - correct length
    if (len(armRsltDF)) == 32019:
        print('Correct Submission Length')
    else:
        print('!! ERROR !! - Incorrect Submission Length')

    def fn_write_submission_file(df, colNames, fileName):

        print(colNames[1])
        forecastCol = colNames[1]

        tempDF = df.copy()

        # Rename the second column to "visitors" as per submission
        tempDF.rename(columns={forecastCol: 'visitors'}, inplace=True)

        #print(tempDF.head())
        tempDF.to_csv(fileName, header=True, index=False, quotechar='"', columns=('id','visitors'))
        print('Wrote file: ' + fileName)

    todayDate = str(dt.date.today().strftime('%Y%m%d'))

    # Float forecast
    exportDF = armRsltDF.copy()
    exportDF
    columns=('id','forecast')
    fn_write_submission_file(armRsltDF, columns, arimaExportDir + todayDate + '_subm_frcst_flt.csv')
